credits: dionysis kokkoris

In [ ]:
# to handle datasets
import pandas as pd
import numpy as np

# navigate folders
from glob import glob
import os

# for plotting
import matplotlib.pyplot as plt
%matplotlib inline

# to open the images
import cv2

# to display all the columns of the dataframe in the notebook
pd.pandas.set_option('display.max_columns', None)
# data preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# evaluate model and separate train and test
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore")
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing import image
from keras.utils import np_utils

In [ ]:
train=pd.read_csv('../input/ranzcr-clip-catheter-line-classification/train.csv')
test=pd.read_csv('../input/ranzcr-clip-catheter-line-classification/sample_submission.csv')
train['images']='../input/ranzcr-clip-catheter-line-classification/train/'+train['StudyInstanceUID']+'.jpg'
test['images']='../input/ranzcr-clip-catheter-line-classification/test/'+test['StudyInstanceUID']+'.jpg'


In [ ]:
train

In [ ]:
y=train[['ETT - Abnormal', 'ETT - Borderline',
       'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline',
       'NGT - Incompletely Imaged', 'NGT - Normal', 'CVC - Abnormal',
       'CVC - Borderline', 'CVC - Normal', 'Swan Ganz Catheter Present']]
y=y.values

In [ ]:
def im_resize(df, n):
    #print(df[n])
    im = cv2.imread(df[n])
    im = cv2.resize(im, (IMAGE_SIZE, IMAGE_SIZE))
    #plt.imshow(im)
    #plt.show()

    return im
def create_dataset(df, image_size):
    # functions creates dataset as required for cnn
    tmp = np.zeros((len(df), image_size, image_size,3), dtype='float32')

    for n in range(0, len(df)):
        im = im_resize(df, n)
        tmp[n] = im
  
    print('Dataset Images shape: {} size: {:,}'.format(tmp.shape, tmp.size))
    return tmp

In [ ]:
IMAGE_SIZE = 150
# X=create_dataset(train['images'],150)

X = np.load('../input/ranzcr-numpy-150/X.npy',mmap_mode="r")


https://stackoverflow.com/questions/9244397/memory-overflow-when-using-numpy-load-in-a-loop/65893798#65893798

In [ ]:
# import gc
# d = dict()
# for o in gc.get_objects():
#     name = type(o).__name__
#     if name not in d:
#         d[name] = 1
#     else:
#         d[name] += 1

# items = d.items()
# #items.sort(key=lambda x:x[1])
# for key, value in items:
#     print(key, value)


In [ ]:
X.shape

In [ ]:
xt=create_dataset(test['images'],150 )
# xt = np.load('../input/ranzcr-numpy-150/xt.npy',mmap_mode="r")
# xt.shape

In [ ]:
IMAGE_SIZE = 150
kernel_size = (3,3)
pool_size= (2,2)
first_filters = 32
second_filters = 64
third_filters = 128

dropout_conv = 0.3
dropout_dense = 0.3

model = Sequential()
model.add(Conv2D(first_filters, kernel_size, activation = 'relu', 
                 input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)))
model.add(Conv2D(first_filters, kernel_size, activation = 'relu'))
#model.add(Conv2D(first_filters, kernel_size, activation = 'relu'))
model.add(MaxPooling2D(pool_size = pool_size)) 
model.add(Dropout(dropout_conv))

model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
#model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
model.add(MaxPooling2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

model.add(Conv2D(third_filters, kernel_size, activation ='relu'))
model.add(Conv2D(third_filters, kernel_size, activation ='relu'))
#model.add(Conv2D(third_filters, kernel_size, activation ='relu'))
model.add(MaxPooling2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(dropout_dense))
model.add(Dense(11, activation = "softmax"))

model.summary()
model.compile(Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X, y,batch_size=32,epochs = 10,validation_split=0.1)


In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
ynew = model.predict_proba(xt)
labels = test.columns[1:12]
test[labels]=ynew
test=test.drop('images',axis=1)
test.to_csv("submission.csv",index=False)